In [ ]:
import geopandas as gpd
import pandas as pd
import xarray as xr
import numpy as np
import pygmt
import os
from datetime import datetime

In [ ]:
pixels_per_tile = 512 * 2
# Light subset of CONUS + Alaska
min_lat = 24
max_lat = 72
min_lon = -180
max_lon = -66

pixels_per_tile = 512 * 2
day_range = 3

In [ ]:
store = f"s3://carbonplan-scratch/firms-data/firms-{datetime.now().strftime('%Y-%m-%d')}.zarr"
store

In [ ]:
base = "https://firms.modaps.eosdis.nasa.gov/usfs/api/area/csv/"
subset_str = f"{min_lon},{min_lat},{max_lon},{max_lat}"
df = gpd.pd.read_csv(f"{base}{key}/VIIRS_SNPP_NRT/{subset_str}/{day_range}/")
df = df[df['confidence'] == 'h']
df['acq_date'] = pd.to_datetime(df['acq_date'])
df['registered'] = 1
df

In [ ]:
def rasterize_frp(df: pd.DataFrame) -> xr.Dataset:
    active = pygmt.xyz2grd(
        data=df[['longitude', 'latitude', 'registered']],
        region=[min_lon, max_lon, min_lat, max_lat],
        spacing="400e",
        duplicate="u",
        registration="p",
    ).chunk({'lon': pixels_per_tile, 'lat': pixels_per_tile})
    active = xr.where(active.notnull(), 1, 0).astype('i1')
    active.attrs['_FillValue'] = 0
    active = active.to_dataset(name='active')
    return active

In [ ]:
burned = rasterize_frp(df)
burned

In [ ]:
burned.to_zarr(
    store,
    encoding={'active': {"write_empty_chunks": False, "dtype": 'i1'}},
    mode="w",
    consolidated=True,
)

## Check sparsity

In [ ]:
import zarr

In [ ]:
root = zarr.open_consolidated(store)
print(f'nchunks: {root["active"].nchunks}')
print(f'nchunks_initiliazed: {root["active"].nchunks_initialized}')
print(root["active"].dtype)

## Generate pyramids

In [ ]:
from ndpyramid import pyramid_reproject
import xarray as xr
import regionmask
import rioxarray
import zarr
import numpy as np
from carbonplan_data.utils import set_zarr_encoding as set_web_zarr_encoding

In [ ]:
output_store = f's3://carbonplan-forest-offsets/fires/firms_nrt/current-firms-hotspots.zarr'
output_store
LEVELS = 6

In [ ]:
ds = xr.open_zarr(store, consolidated=True)
print(ds.active.dtype.str)

In [ ]:
dt = pyramid_reproject(ds.rio.write_crs("EPSG:4326"), levels=LEVELS, resampling="sum")

In [ ]:
for child in dt.children:
    dt[child]['active'] = xr.where(dt[child]['active'] > 0, 1, np.nan)
    dt[child].ds = set_web_zarr_encoding(
        dt[child].ds, codec_config={"id": "zlib", "level": 1}, float_dtype="float32"
    )

In [ ]:
dt.to_zarr(output_store, consolidated=True, mode='w')

## Scratch

In [ ]:
root = zarr.open_consolidated(output_store)
for level in range(LEVELS):
    slevel = str(level)
    print(f'level {slevel}')
    print(f'    nchunks: {root[slevel]["active"].nchunks}')
    print(f'    nchunks_initiliazed level: {root[level]["active"].nchunks_initialized}')

In [ ]:
root["0"]["active"][:]

In [ ]:
ds = xr.open_zarr('s3://carbonplan-forest-offsets/fires/firms_nrt/current-firms-hotspots.zarr/0/')
ds

In [ ]:
ds['active'].plot()

In [ ]:
ds.active.load()

In [ ]:
ds.active.min()